In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.express as px

In [2]:
# 데이터를 불러옵니다.
donation_original = pd.read_csv('./data/donations_dollar.csv')
general_original = pd.read_csv('./data/general_success.csv')

In [6]:
general_original.head()

,Unnamed: 0,chid,url,scrape_date,title,created_date,funds_raised,funds_target,picture,org_picture,...,campaign_hearts,social_share_total,num_recent_donations,recent_time_window,funds_raised_dollar,funds_target_dollar,amount_raised_unattributed_dollar,char_age,success,dum_success
0,0,1.0,https://www.gofundme.com/f/-chris-beaty-memori...,2020-11-03,Chris Beaty Memorial Fund,2020-06-03,175083.0,200000.0,True,False,...,1691.0,9069.0,0.0,48 hours,175083,200000.0,0.0,153,False,0
1,1,2.0,https://www.gofundme.com/f/1-prosent-chance,2020-11-03,1 percent chance,2020-06-21,299042.0,2000000.0,True,False,...,542.0,1882.0,0.0,48 hours,33792,226000.0,0.0,135,False,0
2,2,3.0,https://www.gofundme.com/f/100-computers-in-10...,2020-11-04,MELS needs #100ComputersIn100Days,2020-06-19,13845.0,30000.0,True,True,...,119.0,383.0,0.0,48 hours,13845,30000.0,8196.0,138,False,0
3,3,4.0,https://www.gofundme.com/f/100-to-victims-of-t...,2020-11-03,100% to Victims of Taal Volcano in Philippines,2020-01-16,25.0,5000.0,True,True,...,1.0,0.0,0.0,48 hours,25,5000.0,0.0,292,False,0
4,4,5.0,https://www.gofundme.com/f/100k-for-center-for...,2020-11-04,$100k for Center for Family Life,2020-11-02,13950.0,100000.0,True,False,...,17.0,0.0,16.0,48 hours,13950,100000.0,1100.0,2,False,0


# 작업을 위한 테이블 편집

In [8]:
#원하는 컬럼만 추출
general = general_original[['chid','title','created_date','funds_raised_dollar', 'funds_target_dollar', 'amount_raised_unattributed_dollar', 'picture', 'org_picture', 'campaign_hearts', 'num_recent_donations', 'total_donations', 'success']]
general.columns = ['chid','title','created_date','funds_raised_d', 'funds_target_d', 'amount_anonymous', 'picture', 'org_picture', 'campaign_hearts', 'num_recent_donations', 'total_donations', 'success']
general.head()

,chid,title,created_date,funds_raised_d,funds_target_d,amount_anonymous,picture,org_picture,campaign_hearts,num_recent_donations,total_donations,success
0,1.0,Chris Beaty Memorial Fund,2020-06-03,175083,200000.0,0.0,True,False,1691.0,0.0,1705.0,False
1,2.0,1 percent chance,2020-06-21,33792,226000.0,0.0,True,False,542.0,0.0,602.0,False
2,3.0,MELS needs #100ComputersIn100Days,2020-06-19,13845,30000.0,8196.0,True,True,119.0,0.0,125.0,False
3,4.0,100% to Victims of Taal Volcano in Philippines,2020-01-16,25,5000.0,0.0,True,True,1.0,0.0,1.0,False
4,5.0,$100k for Center for Family Life,2020-11-02,13950,100000.0,1100.0,True,False,17.0,16.0,17.0,False


In [10]:
# 원하는 컬럼만 추출
donation = donation_original[['chid','is_anonymous','created_at', 'amount_dollar']]

In [57]:
donation.columns = ['chid','is_anonymous','donated_at', 'amount_d']

In [12]:
# donated_at 타입변경 및 시간스탬프 삭제
pd.options.mode.chained_assignment = None
donation.loc[:,'donated_at'] = pd.to_datetime(donation.loc[:,'donated_at']).dt.date

In [13]:
donation.dtypes

chid              int64
is_anonymous       bool
donated_at       object
amount_d        float64
dtype: object

In [14]:
# general과 donation 테이블 inner join
df_1 = pd.merge(general, donation, how = 'inner', on = 'chid')
df_1.dtypes

chid                    float64
title                    object
created_date             object
funds_raised_d            int64
funds_target_d          float64
amount_anonymous        float64
picture                    bool
org_picture                bool
campaign_hearts         float64
num_recent_donations    float64
total_donations         float64
success                    bool
is_anonymous               bool
donated_at               object
amount_d                float64
dtype: object

In [15]:
df_1[["created_date", "donated_at"]] = df_1[["created_date", "donated_at"]].apply(pd.to_datetime)
df_1.dtypes

chid                           float64
title                           object
created_date            datetime64[ns]
funds_raised_d                   int64
funds_target_d                 float64
amount_anonymous               float64
picture                           bool
org_picture                       bool
campaign_hearts                float64
num_recent_donations           float64
total_donations                float64
success                           bool
is_anonymous                      bool
donated_at              datetime64[ns]
amount_d                       float64
dtype: object

In [16]:
# 펀딩 성공여부를 알 수 있는 불리언 타입의 'is_successful' 컬럼 추가
df = df_1.assign(is_successful = df_1['funds_raised_d'] >= df_1['funds_target_d'],
                date_diff = df_1['donated_at'] - df_1['created_date'])
df.head()

,chid,title,created_date,funds_raised_d,funds_target_d,amount_anonymous,picture,org_picture,campaign_hearts,num_recent_donations,total_donations,success,is_anonymous,donated_at,amount_d,is_successful,date_diff
0,1.0,Chris Beaty Memorial Fund,2020-06-03,175083,200000.0,0.0,True,False,1691.0,0.0,1705.0,False,False,2020-10-14,480.0,False,133 days
1,1.0,Chris Beaty Memorial Fund,2020-06-03,175083,200000.0,0.0,True,False,1691.0,0.0,1705.0,False,False,2020-09-15,10.0,False,104 days
2,1.0,Chris Beaty Memorial Fund,2020-06-03,175083,200000.0,0.0,True,False,1691.0,0.0,1705.0,False,False,2020-09-10,50.0,False,99 days
3,1.0,Chris Beaty Memorial Fund,2020-06-03,175083,200000.0,0.0,True,False,1691.0,0.0,1705.0,False,False,2020-09-09,100.0,False,98 days
4,1.0,Chris Beaty Memorial Fund,2020-06-03,175083,200000.0,0.0,True,False,1691.0,0.0,1705.0,False,False,2020-09-08,100.0,False,97 days


In [17]:
# timedelta인 date_diff를 숫자형(float)으로 변환 (그래프 x축에 두기 위해)
df['date_diff'] = df['date_diff'].apply(lambda x:x.days)

In [18]:
df.dtypes

chid                           float64
title                           object
created_date            datetime64[ns]
funds_raised_d                   int64
funds_target_d                 float64
amount_anonymous               float64
picture                           bool
org_picture                       bool
campaign_hearts                float64
num_recent_donations           float64
total_donations                float64
success                           bool
is_anonymous                      bool
donated_at              datetime64[ns]
amount_d                       float64
is_successful                     bool
date_diff                        int64
dtype: object

In [19]:
# 랜덤으로 chid 693번 펀딩의 정보 확인
df.loc[df['chid'] == 693,:].head()

,chid,title,created_date,funds_raised_d,funds_target_d,amount_anonymous,picture,org_picture,campaign_hearts,num_recent_donations,total_donations,success,is_anonymous,donated_at,amount_d,is_successful,date_diff
403212,693.0,Black Leadership Initiative,2020-06-26,15847,15000.0,9970.0,True,True,186.0,0.0,209.0,True,False,2020-07-21,100.0,True,25
403213,693.0,Black Leadership Initiative,2020-06-26,15847,15000.0,9970.0,True,True,186.0,0.0,209.0,True,True,2020-07-13,25.0,True,17
403214,693.0,Black Leadership Initiative,2020-06-26,15847,15000.0,9970.0,True,True,186.0,0.0,209.0,True,False,2020-07-11,20.0,True,15
403215,693.0,Black Leadership Initiative,2020-06-26,15847,15000.0,9970.0,True,True,186.0,0.0,209.0,True,False,2020-07-10,500.0,True,14
403216,693.0,Black Leadership Initiative,2020-06-26,15847,15000.0,9970.0,True,True,186.0,0.0,209.0,True,False,2020-07-09,25.0,True,13


In [20]:
df.head()

,chid,title,created_date,funds_raised_d,funds_target_d,amount_anonymous,picture,org_picture,campaign_hearts,num_recent_donations,total_donations,success,is_anonymous,donated_at,amount_d,is_successful,date_diff
0,1.0,Chris Beaty Memorial Fund,2020-06-03,175083,200000.0,0.0,True,False,1691.0,0.0,1705.0,False,False,2020-10-14,480.0,False,133
1,1.0,Chris Beaty Memorial Fund,2020-06-03,175083,200000.0,0.0,True,False,1691.0,0.0,1705.0,False,False,2020-09-15,10.0,False,104
2,1.0,Chris Beaty Memorial Fund,2020-06-03,175083,200000.0,0.0,True,False,1691.0,0.0,1705.0,False,False,2020-09-10,50.0,False,99
3,1.0,Chris Beaty Memorial Fund,2020-06-03,175083,200000.0,0.0,True,False,1691.0,0.0,1705.0,False,False,2020-09-09,100.0,False,98
4,1.0,Chris Beaty Memorial Fund,2020-06-03,175083,200000.0,0.0,True,False,1691.0,0.0,1705.0,False,False,2020-09-08,100.0,False,97


In [21]:
# 랜덤으로 chid 329번 펀딩의 정보 확인
df.loc[df['chid'] == 329,:].head()

,chid,title,created_date,funds_raised_d,funds_target_d,amount_anonymous,picture,org_picture,campaign_hearts,num_recent_donations,total_donations,success,is_anonymous,donated_at,amount_d,is_successful,date_diff
190621,329.0,Karen Nascembeni,2020-04-11,55435,50000.0,0.0,True,False,634.0,0.0,638.0,False,True,2020-10-29,10.0,True,201
190622,329.0,Karen Nascembeni,2020-04-11,55435,50000.0,0.0,True,False,634.0,0.0,638.0,False,False,2020-10-29,50.0,True,201
190623,329.0,Karen Nascembeni,2020-04-11,55435,50000.0,0.0,True,False,634.0,0.0,638.0,False,False,2020-10-29,50.0,True,201
190624,329.0,Karen Nascembeni,2020-04-11,55435,50000.0,0.0,True,False,634.0,0.0,638.0,False,False,2020-10-29,100.0,True,201
190625,329.0,Karen Nascembeni,2020-04-11,55435,50000.0,0.0,True,False,634.0,0.0,638.0,False,False,2020-07-25,100.0,True,105


# 일자별 기부횟수, 기부액 총합 살펴보기
## 기부횟수
### 일자별 기부 발생 수

In [40]:
# 날짜별 donation chid 카운트하여 일별 기부 수 구함
df_day_numbers = df.groupby('created_date')[["chid"]].count().reset_index()
df_day_numbers.head()

,created_date,chid
0,2019-11-03,2016
1,2019-11-04,3683
2,2019-11-05,1755
3,2019-11-06,3552
4,2019-11-07,4374


In [56]:
# 일자별 발생한 기부횟수
fig = px.bar(df_day_numbers, x='created_date', y='chid', height=500, title='일자별 기부 발생 수')
# fig.show()

### 요일별 기부 발생 수

In [47]:
# 요일 데이터 추가
df_day_numbers['weekday'] = df_day_numbers['created_date'].dt.dayofweek

In [48]:
# 요일별 발생한 기부횟수 구하기
# 0: 월요일, 6: 일요일
df_wday_numbers = df_day_numbers.groupby('weekday')[["chid"]].sum().sort_values('weekday', ascending=True).reset_index()

In [50]:
from pandas import Series, DataFrame # 급 import

In [52]:
# 숫자 요일을 문자 요일로도 만들어줌
df_wday_numbers['wday_name'] = Series(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"], index=[0, 1, 2, 3, 4, 5, 6])
df_wday_numbers

,weekday,chid,wday_name
0,0,368806,Monday
1,1,326426,Tuesday
2,2,318133,Wednesday
3,3,329103,Thursday
4,4,308981,Friday
5,5,249157,Saturday
6,6,291071,Sunday


In [55]:
# 요일별 발생한 기부횟수
fig = px.bar(df_wday_numbers, x='wday_name', y='chid', height=500, title='요일별 기부 발생 수')
# fig.show()

## 기부액 총합
### 일자별 기부액 총합

In [25]:
df_day_amount = df.groupby('created_date')[["funds_raised_d", "funds_target_d"]].sum().reset_index()
df_day_amount['funds_raised_d'] = round(df_day_amount['funds_raised_d'])
df_day_amount['funds_target_d'] = round(df_day_amount['funds_target_d'])
df_day_amount.head()

,created_date,funds_raised_d,funds_target_d
0,2019-11-03,129331358,166880000.0
1,2019-11-04,339794866,489283830.0
2,2019-11-05,239493924,356661000.0
3,2019-11-06,244217040,269647000.0
4,2019-11-07,363678288,208313414.0


In [54]:
# 일자별 기부액 총합
fig = px.line(df_day_amount, x='created_date', y='funds_raised_d', height=500, title='일자별 기부액 총합')
# fig.show()

In [ ]:
# 2020-03-15 에 기부된 금액, 주로 어떤 프로젝트로 갔을까? 코로나19 판데믹 선언과 관련 있을까?
df_0315 = df[df['donated_at']=='2020-03-15'].groupby('title')[["amount_d"]].sum().sort_values('amount_d', ascending=False).reset_index()
df_0315.head(10)

### 케이스 톺아보기 (전체 모금 금액 1st로 높았던 2020-03-15 best 건)
**Fight4Fiachra**
- 3.14에 open한 프로젝트
- 코로나19 관련 X 
- 희귀한 백혈병 환자에 대한 모금
- 3.19부터 11.11까지 9회에 걸친 업데이트로 환자 상태 및 상황 공유
- 펀딩이 열린 8개월 전의 모금액수가 압도적으로 많아 보이며, 현재까지도 (가장 최근 모금 9일 전) 모금 진행중

**Orange County United Way Pandemic Relief Fund**
- 코로나19 관련

**Un Respiro per Olbia - Emergenza COVID19**
- 코로나19 관련

**#insieme per la terapia intensiva di CISANELLO**
- 코로나19 관련

**Help Pomona students impacted by COVID-19**
- 코로나19 관련

In [ ]:
# 2020-06-01 에 기부된 금액, 주로 어떤 프로젝트로 갔을까? Black Lives Matter와 관련 있을까?
df_0601 = df[df['donated_at']=='2020-06-01'].groupby('title')[["amount_d"]].sum().sort_values('amount_d', ascending=False).reset_index()
df_0601.head(10)

### 케이스 톺아보기 (전체 모금 금액 3rd로 높았던 2020-06-01 best 건)
(참고) On May 25, 2020, George Floyd, a 46-year-old black man, was killed in Minneapolis, Minnesota, while being arrested for allegedly using a counterfeit bill.

**Supporting Robbie McEachern and Family**
- 암에 걸린 남성에 대한 펀딩
- 2020-05-31에 업로드되었고, 다음날인 2020-06-01에 압도적으로 많은 기부 들어옴
- 총 기부금액(11.24 홈피 기준) $167,980. 전체 금액 기준으로 봐도 첫날에 25% 이상 들어온 것 확인
- 2020-06-02에 감사 업데이트, 2020-06-19에 사망했다는 내용의 업데이트(사망일 6.18)가 있음

**Orange County United Way Pandemic Relief Fund**
- 코로나19 케이스와 동일 (스테디셀러)

**Matt Gilliam & Family Memorial Fund**
- 사망한 가장의 가족을 위한 펀딩
- 6.1에 시작했고 대부분의 금액이 6.5까지 모금되어 6.5에 감사인사 업데이트
- 하지만 현재까지 목표금액인 150000을 채우지는 못함, 그래도 꽤 많은 금액(131020)

**#TeamDessino**
- 사망하게 된 한 남성에 대한 스토리
- 인물에 대한 맥락 소개가 자세함
- 5.31에 업로드했고 6.2에 업데이트 2회 있었음
- 목표금액 100000를 101075로 달성

**Help Us Rebuild Sunny Optometry Destroyed by Riots**
- 폭도들(?)이 병원을 침입하여 각종 장비들을 훼손하고 약탈한 상황으로 보임
- 전체 250000 중 87531만 달성
- 장문의 스토리와 유튜브 링크도 몇 개 들어가있음

# 타이틀 특정 문구(예. COVID, Coronavirus, Black 등) 별로 콘텐츠 분류 가능한지 확인
해보지는 않았고, 시도가 가능하다 정도만 남겨둠

## 코로나19 관련
- COVID
- Corona

In [ ]:
df[df['title'].str.contains('COVID')].groupby('title')[["amount_d"]].sum().sort_values('amount_d', ascending=False).reset_index()

In [ ]:
df[df['title'].str.contains('Corona')].groupby('title')[["amount_d"]].sum().sort_values('amount_d', ascending=False).reset_index()

## BlackLivesMatter 관련
- FLOYD
- Matter
- MATTER

In [ ]:
df[df['title'].str.contains('FLOYD')].groupby('title')[["amount_d"]].sum().reset_index()

In [ ]:
df[df['title'].str.contains('Matter')].groupby('title')[["amount_d"]].sum().reset_index()

In [ ]:
df[df['title'].str.contains('MATTER')].groupby('title')[["amount_d"]].sum().reset_index()

# 분야별 기부횟수, 기부액 비교
## 앗 분야 데이터가 왜 그 어디에도 없지?

In [69]:
# 데이터를 불러옵니다.
donation_original = pd.read_csv('./data/donations_dollar.csv')
general_original = pd.read_csv('./data/general_success.csv')

In [62]:
# 좀 더 불러옵니다.
organizer_original = pd.read_csv('./data/organizer_dollar.csv')
updates_original = pd.read_csv('./data/updates_new.csv')
comments_original = pd.read_csv('./data/comments_dollar.csv')

In [64]:
organizer_original.tail()

,chid,org_id,org_name,org_type,location,raised,num_donations,currency,raised_dollar
8199,3903,8199,Kelly Culver,Beneficiary,NaN,NaN,NaN,dollar,NaN
8200,3904,8200,Sarah Manoucheri,Organizer,"Glen Cove, NY",NaN,NaN,dollar,NaN
8201,3904,8201,Ineta Yaghoubian,Beneficiary,NaN,NaN,NaN,dollar,NaN
8202,3905,8202,Family And Friends Of Carmella Rossi,Organizer,"Buffalo, NY",NaN,NaN,dollar,NaN
8203,3905,8203,Anthony Rossi,Beneficiary,NaN,NaN,NaN,dollar,NaN


In [67]:
updates_original.head()

,chid,update_id,author,author_type,user_profile_url,text,photos,created_at
0,1,24893752,Organizer,ORGANIZER,NaN,Thank you to everyone who donated to Chris’ Go...,False,2020-06-18 16:24:00+00:00
1,3,25518194,Organizer,ORGANIZER,https://graph.facebook.com/v7.0/10156402123230...,We have raised over $12k for our computers! Th...,False,2020-08-30 10:27:00+00:00
2,3,25184752,Organizer,ORGANIZER,https://graph.facebook.com/v7.0/10156402123230...,No matter what school looks like in September ...,False,2020-07-22 16:36:00+00:00
3,3,25010100,Organizer,ORGANIZER,https://graph.facebook.com/v7.0/10156402123230...,Our campaign is 25% there! Thank you so much t...,False,2020-07-02 11:18:00+00:00
4,3,24922594,Organizer,ORGANIZER,https://graph.facebook.com/v7.0/10156402123230...,We&#039;ve raised $4k towards our goal since F...,True,2020-06-22 11:36:00+00:00


In [68]:
comments_original.head()

,chid,comment_id,name,timestamp,profile_url,photos,amount,is_anonymous,created_at,comment,currency,amount_dollar
0,1,0,NIAH FULFORD,2020-08-17 00:29:34+00:00,NaN,True,1000.0,False,2020-08-17 05:27:14+00:00,This donation is from the Black Alum Council o...,dollar,1000.0
1,1,1,Kurt Nauenburg,2020-07-05 10:17:37+00:00,NaN,NaN,50.0,False,2020-07-05 15:12:15+00:00,Justice for Chris!,dollar,50.0
2,1,2,Alfonzo Thurman,2020-06-29 14:19:29+00:00,NaN,NaN,50.0,False,2020-06-29 19:15:16+00:00,I didn't know Chris personally but he clearly ...,dollar,50.0
3,1,3,Courtney Whistler,2020-06-27 17:14:21+00:00,NaN,NaN,770.0,False,2020-06-27 22:12:11+00:00,Doug Bright/ Binkley's Kitchen and Bar,dollar,770.0
4,1,4,Kevin Emberton,2020-06-13 19:15:44+00:00,NaN,NaN,100.0,False,2020-06-14 00:14:57+00:00,Thoughts and prayers to family,dollar,100.0


In [72]:
general_original.head()

,Unnamed: 0,chid,url,scrape_date,title,created_date,funds_raised,funds_target,picture,org_picture,...,campaign_hearts,social_share_total,num_recent_donations,recent_time_window,funds_raised_dollar,funds_target_dollar,amount_raised_unattributed_dollar,char_age,success,dum_success
0,0,1.0,https://www.gofundme.com/f/-chris-beaty-memori...,2020-11-03,Chris Beaty Memorial Fund,2020-06-03,175083.0,200000.0,True,False,...,1691.0,9069.0,0.0,48 hours,175083,200000.0,0.0,153,False,0
1,1,2.0,https://www.gofundme.com/f/1-prosent-chance,2020-11-03,1 percent chance,2020-06-21,299042.0,2000000.0,True,False,...,542.0,1882.0,0.0,48 hours,33792,226000.0,0.0,135,False,0
2,2,3.0,https://www.gofundme.com/f/100-computers-in-10...,2020-11-04,MELS needs #100ComputersIn100Days,2020-06-19,13845.0,30000.0,True,True,...,119.0,383.0,0.0,48 hours,13845,30000.0,8196.0,138,False,0
3,3,4.0,https://www.gofundme.com/f/100-to-victims-of-t...,2020-11-03,100% to Victims of Taal Volcano in Philippines,2020-01-16,25.0,5000.0,True,True,...,1.0,0.0,0.0,48 hours,25,5000.0,0.0,292,False,0
4,4,5.0,https://www.gofundme.com/f/100k-for-center-for...,2020-11-04,$100k for Center for Family Life,2020-11-02,13950.0,100000.0,True,False,...,17.0,0.0,16.0,48 hours,13950,100000.0,1100.0,2,False,0


In [73]:
donation_original.head()

,chid,donation_id,amount,is_anonymous,name,created_at,profile_url,currency,amount_dollar
0,1,0,480.0,False,Pedal For Peace Bike Ride T-shirt Sales,2020-10-14 11:43:12+00:00,NaN,dollar,480.0
1,1,1,10.0,False,Jacquelin Pabon,2020-09-15 04:51:30+00:00,NaN,dollar,10.0
2,1,2,50.0,False,Mindy Bemis,2020-09-10 00:52:25+00:00,NaN,dollar,50.0
3,1,3,100.0,False,Dana Severs,2020-09-09 05:03:36+00:00,NaN,dollar,100.0
4,1,4,100.0,False,Linsey Fraser,2020-09-08 04:05:18+00:00,NaN,dollar,100.0
